In [1]:
import pandas as pd
import numpy as np
import glob

In [3]:
# 处理计算学业水平分，得到Part1.xlsx
file_path = "data\\牢数据2班.xlsx"

total_score = pd.read_excel(file_path, sheet_name='Sheet1')

pe1 = pd.read_excel(file_path, sheet_name='体育1')
pe2 = pd.read_excel(file_path, sheet_name='体育2')

df = total_score.copy()
pe1_lessons = pe1.columns.drop(['序号', '学号', '姓名']).tolist()
pe2_lessons = pe2.columns.drop(['序号', '学号', '姓名']).tolist()

def find_pe_score_apply(row, lessons):
    for col in lessons:
        val = row[col]
        if isinstance(val, str) and val.strip() == '--':
            val = float('nan')
        if pd.notna(val):
            return val
    return float('nan')

pe1['final_score'] = pe1.apply(find_pe_score_apply, axis=1, lessons=pe1_lessons)
pe2['final_score'] = pe2.apply(find_pe_score_apply, axis=1, lessons=pe2_lessons)

df = df.merge(pe1[['学号', 'final_score']].rename(columns={'final_score': 'pe1_score'}), on='学号', how='left')
df = df.merge(pe2[['学号', 'final_score']].rename(columns={'final_score': 'pe2_score'}), on='学号', how='left')

def calc(row):
        pe1 = pd.to_numeric(row['pe1_score'], errors='coerce')
        pe2 = pd.to_numeric(row['pe2_score'], errors='coerce')
        total = row['均分'] * (row['总学分'] - 1) # 减去国家安全教育
        # 体育分遇到特殊情况特殊处理
        if pd.isna(pe1) and pd.isna(pe2):
            return total / (row['总学分'] - 1)
        elif pd.isna(pe1): # 假设没选体育，具体情况具体分析
            return (total - pe2) / (row['总学分'] - 2)
        elif pd.isna(pe2): # 假设没选体育，具体情况具体分析
            return (total - pe1) / (row['总学分'] - 2)
        else:
            return (total - pe1 - pe2) / (row['总学分'] - 3)
        
df['除去体育分数'] = df.apply(calc, axis=1)

df['学业水平分'] = df['除去体育分数'] * 0.6
df['体育分数'] = (df['pe1_score'].fillna(0) + df['pe2_score'].fillna(0)) / 100 

df.to_csv('data/final_data.csv', index=False, encoding='utf-8')

with pd.ExcelWriter('data/Part1.xlsx') as writer:
    df.to_excel(writer, sheet_name='scores', index=False)

In [ ]:
file_paths = glob.glob('data/deyu_score/*.xlsx')
test = pd.read_excel(file_paths[0])
df = pd.DataFrame()

for file in file_paths:
    cur = pd.read_excel(file)
    number = file.split('\\')[1].split('德育')[0]
    # print(number)
    df[number] = cur['德育评分']

df['学号'] = test['学号']
df.set_index('学号', inplace=True)


def calc_row(row):
    # 除去自己分数后的所有分数
    scores = [row[col] for col in df.columns if col != str(row.name)]
    # 计算平均分
    return pd.Series(scores).mean()

df['final'] = df.apply(calc_row, axis=1)

df.to_excel('data/德育汇总.xlsx', sheet_name='德育汇总')

missing_index = [idx for idx in df.index if str(idx) not in df.columns]
print("缺少德育评分的学号：", missing_index)

# 综合到Part1表格
final_score = pd.read_excel('data/Part1.xlsx', index_col='学号')
final_score.drop(columns=['序号'], inplace=True, axis=0)
final_score = final_score.join(df['final'])
final_score.rename(columns={'final': '德育成绩'}, inplace=True)

with pd.ExcelWriter('data/Part2.xlsx') as writer:
    final_score.to_excel(writer, sheet_name='scores', index=True)

缺少德育评分的学号： []
